In [1]:
import numpy as np

from scipy import stats

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

In [2]:
X = np.sort(np.random.rand(5,2)*20+5)
Y = X[:,0]*2 + X[:,0]*3

In [3]:
TRAIN_TEST_RATIO = 0.8
M = X.shape[0]
M_train = int(np.ceil(0.8*M))

In [4]:
scaler = MinMaxScaler()
scaler = scaler.fit(X)
#X = scaler.transform(X)

indices = np.random.permutation(X.shape[0])
train_idx, test_idx = indices[:M_train], indices[M_train:]
x_train, x_test = X[train_idx, :], X[test_idx, :]
y_train, y_test = Y[train_idx], Y[test_idx]

In [5]:
def calculate_prior(y):
    return np.array([(y==0).sum(axis=0), (y==1).sum(axis=0)])/y.size
def calculate_theta(x):
    m, n = x.shape
    mean = np.empty((0, n))
    var = np.empty((0, n))
    for i in range(m):
        mean = np.append(mean, x[:i+1].mean(axis=0).reshape(1,-1), axis=0)
        var = np.append(var, x[:i+1].var(axis=0).reshape(1,-1), axis=0)
    return mean, var
def calculate_posterior(mean, var, x):
    m = x.shape[0]
    m_train = mean.shape[0]
    prior = np.arange(1, m_train+1)/m_train
    posterior = np.empty((0, m))
    for i in range(m_train):
        post = (stats.norm(mean[i], np.sqrt(var[i])).pdf(x).prod(axis=1)*prior[i]).reshape(1,-1)
        posterior = np.append(posterior, post, axis=0)
    posterior = posterior/np.nansum(posterior, axis=0)
    return posterior
def mean_squared_error(y, y_train, posterior):
    return np.round(np.square(np.nansum(posterior*y_train, axis=0)-y).mean(axis=0), 2)

In [6]:
a = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12]])
a = a/np.nansum(a, axis=0)
np.nansum(a, axis=0)

array([1., 1., 1., 1.])

In [7]:
mean, var = calculate_theta(x_train)

In [8]:
posterior = calculate_posterior(mean, var, x_train)
mean_squared_error(y_train, y_train, posterior)

C:\Users\smrut\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1740: RuntimeWarning: divide by zero encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
C:\Users\smrut\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1740: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
C:\Users\smrut\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:899: RuntimeWarning: invalid value encountered in greater_equal
  return (a <= x) & (x <= b)
C:\Users\smrut\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:899: RuntimeWarning: invalid value encountered in less_equal
  return (a <= x) & (x <= b)


0.0

In [9]:
posterior

array([[           nan,            nan,            nan,            nan],
       [6.47777310e-01, 5.59891215e-01, 1.05633859e-44, 6.47432712e-02],
       [1.38078382e-01, 1.85480907e-01, 5.07780593e-01, 3.35909906e-01],
       [2.14144308e-01, 2.54627878e-01, 4.92219407e-01, 5.99346823e-01]])

In [10]:
np.nansum(posterior, axis=0)

array([1., 1., 1., 1.])

In [11]:
y_train

array([39.93157379, 72.80335722, 69.14664151, 54.95320343])

In [12]:
posterior[1,:] * y_train

array([2.58667675e+01, 4.07619601e+01, 7.30422656e-43, 3.55785015e+00])

In [13]:
posterior*y_train

array([[           nan,            nan,            nan,            nan],
       [2.58667675e+01, 4.07619601e+01, 7.30422656e-43, 3.55785015e+00],
       [5.51368708e+00, 1.35036327e+01, 3.51113226e+01, 1.84593254e+01],
       [8.55111924e+00, 1.85377644e+01, 3.40353189e+01, 3.29360279e+01]])

In [14]:
posterior = calculate_posterior(mean, var, x_test)
mean_squared_error(y_test, y_train, posterior)

372.36

In [17]:
y_test

array([44.92446419])

In [16]:
np.nansum(posterior*y_train, axis=0)

array([39.93157379, 72.80335722, 69.14664151, 54.95320343])

In [38]:
(posterior*y_train.reshape(-1,1))[1:,].sum()

60.31428368307324

In [ ]:
np.nansum(posterior*y_train.reshape(-1, 1), axis=0)